# House Price Predictor

In [51]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,r2_score
from torch.utils.data import TensorDataset, DataLoader

In [52]:
# -----------------------------
# 1. Load the dataset
# -----------------------------
# Replace 'house_prices.csv' with the path to your dataset.
data = pd.read_csv('train.csv')
data
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1000 non-null   int64  
 1   MSSubClass     1000 non-null   int64  
 2   MSZoning       1000 non-null   object 
 3   LotFrontage    827 non-null    float64
 4   LotArea        1000 non-null   int64  
 5   Street         1000 non-null   object 
 6   Alley          65 non-null     object 
 7   LotShape       1000 non-null   object 
 8   LandContour    1000 non-null   object 
 9   Utilities      1000 non-null   object 
 10  LotConfig      1000 non-null   object 
 11  LandSlope      1000 non-null   object 
 12  Neighborhood   1000 non-null   object 
 13  Condition1     1000 non-null   object 
 14  Condition2     1000 non-null   object 
 15  BldgType       1000 non-null   object 
 16  HouseStyle     1000 non-null   object 
 17  OverallQual    1000 non-null   int64  
 18  OverallCo

In [53]:
# -----------------------------
# 2. Data Cleaning
# -----------------------------

target='SalePrice' # Define the target column

# Identify categorical columns (if they are not already type 'category')
categorical_cols = data.select_dtypes(include=['object', 'category']).columns

# One-hot encode these columns (original columns are dropped automatically)
data = pd.get_dummies(data, columns=categorical_cols, dtype=int,drop_first=True)

clean_numeric_cols = [col for col in data if data[col].isna().sum() == 0]
data_clean = data[clean_numeric_cols]
print(data_clean.columns)

# Ensure that the target column 'price' is present.
if 'SalePrice' not in data_clean.columns:
    raise ValueError("The target column 'price' is not present in the complete numeric data.")

Index(['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       ...
       'SaleType_ConLI', 'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth',
       'SaleType_WD', 'SaleCondition_AdjLand', 'SaleCondition_Alloca',
       'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=229)


In [54]:
# -----------------------------
# 3. Feature Selection
# -----------------------------
# Compute the correlation matrix using only the cleaned numeric data.
corr_matrix = data_clean.corr()

# Compute absolute correlations of features with the target and drop the target itself.
target_corr = corr_matrix['SalePrice'].drop('SalePrice').abs().sort_values(ascending=False)

# Select only the top 4 features with the highest correlation with 'SalesPrice'
topN_features = target_corr.head(15).index
print("Selected top N features:", list(topN_features))

# Save the feature column names for reindexing later.
train_columns = data_clean.drop(columns=[target]).columns

# Define input features (X) and target variable (y).
X = data_clean[topN_features].values
y = data_clean['SalePrice'].values.reshape(-1, 1)

Selected top N features: ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'ExterQual_TA', 'TotRmsAbvGrd', 'FullBath', 'KitchenQual_TA', 'YearBuilt', 'YearRemodAdd', 'Foundation_PConc', 'Fireplaces', 'BsmtQual_TA']


In [55]:
# -----------------------------
# 4. Data Preprocessing
# -----------------------------
# Split the data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features to improve training stability.
scalerX = StandardScaler()
X_train = scalerX.fit_transform(X_train)
X_test = scalerX.transform(X_test)

scalerY = StandardScaler()
y_train = scalerY.fit_transform(y_train)
y_test = scalerY.transform(y_test)

In [56]:
# Convert the numpy arrays to PyTorch tensors.
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create a TensorDataset and DataLoader for batch processing.
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [57]:
# -----------------------------
# 5. Define the Neural Network Model
# -----------------------------
class HousePriceModel(nn.Module):
    def __init__(self, input_dim):
        super(HousePriceModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # Output layer for regression
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HousePriceModel(input_dim=X_train.shape[1]).to(device)

In [58]:
# -----------------------------
# 6. Set Up Loss Function and Optimizer
# -----------------------------
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# -----------------------------
# 7. Train the Model
# -----------------------------
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * batch_X.size(0)
    
    epoch_loss = running_loss / len(train_dataset)
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

Epoch [10/1000], Loss: 0.1408
Epoch [20/1000], Loss: 0.1178
Epoch [30/1000], Loss: 0.1021
Epoch [40/1000], Loss: 0.0919
Epoch [50/1000], Loss: 0.0813
Epoch [60/1000], Loss: 0.0757
Epoch [70/1000], Loss: 0.0648
Epoch [80/1000], Loss: 0.0561
Epoch [90/1000], Loss: 0.0548
Epoch [100/1000], Loss: 0.0521
Epoch [110/1000], Loss: 0.0369
Epoch [120/1000], Loss: 0.0350
Epoch [130/1000], Loss: 0.0330
Epoch [140/1000], Loss: 0.0290
Epoch [150/1000], Loss: 0.0275
Epoch [160/1000], Loss: 0.0257
Epoch [170/1000], Loss: 0.0267
Epoch [180/1000], Loss: 0.0239
Epoch [190/1000], Loss: 0.0220
Epoch [200/1000], Loss: 0.0230
Epoch [210/1000], Loss: 0.0196
Epoch [220/1000], Loss: 0.0190
Epoch [230/1000], Loss: 0.0193
Epoch [240/1000], Loss: 0.0221
Epoch [250/1000], Loss: 0.0160
Epoch [260/1000], Loss: 0.0151
Epoch [270/1000], Loss: 0.0166
Epoch [280/1000], Loss: 0.0157
Epoch [290/1000], Loss: 0.0156
Epoch [300/1000], Loss: 0.0137
Epoch [310/1000], Loss: 0.0139
Epoch [320/1000], Loss: 0.0151
Epoch [330/1000],

In [59]:
# 8. Evaluate the Model
# -----------------------------
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor.to(device))
    test_loss = criterion(predictions, y_test_tensor.to(device)).item()
    print("Test Mean Squared Error:", test_loss)


# Optionally, to evaluate using scikit-learn's MSE:
predictions_np = predictions.cpu().numpy()
predictions_np = scalerY.inverse_transform(predictions_np)
y_test=y_test_tensor.cpu().numpy()
y_test = scalerY.inverse_transform(y_test_tensor)

mse = mean_squared_error(y_test, predictions_np)
r2=r2_score(y_test, predictions_np)
print("Test MSE (scikit-learn):", mse)
print("Test R2:", r2)
#Test Mean Squared Error: 935741376.0

Test Mean Squared Error: 0.23265963792800903
Test MSE (scikit-learn): 1623653543.4024796
Test R2: 0.6112133934868966


In [60]:
# Load the test.csv file (which lacks SalePrice but includes an 'Id' column)
test_df = pd.read_csv('test.csv')
ids = test_df['Id']

# One-hot encode the test data using the same categorical columns as before
test_encoded = pd.get_dummies(test_df, columns=categorical_cols, drop_first=True)

# Reindex to match the training features
test_encoded = test_encoded.reindex(columns=train_columns, fill_value=0)
test_encoded = test_encoded[topN_features]  # Select the same top features
test_encoded = test_encoded.astype(float)
X_test_new = scalerX.transform(test_encoded)

X_test_tensor_new = torch.tensor(X_test_new, dtype=torch.float32)
model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor_new.to(device))
    test_predictions_np = test_predictions.cpu().numpy()

# Inverse transform predictions to get SalePrice in the original scale.
test_predictions_unscaled = scalerY.inverse_transform(test_predictions_np)

submission_df = pd.DataFrame({
    'ID': ids.astype(int),
    'SALEPRICE': test_predictions_unscaled.flatten().astype(float)
})


print("\nSubmission Preview:")
print(submission_df.head())

# Optionally, export to CSV:
submission_df.to_csv('predictions2.csv', index=False)


Submission Preview:
     ID      SALEPRICE
0  1001  107163.757812
1  1002   58597.085938
2  1003  247260.796875
3  1004  180346.078125
4  1005  212003.453125


/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
